CaSSLe : prepare the base ssl model then divide the data set into tasks and then train the model on each task while saving encoder (frozen) after each task
during training Train the current model (student) using two losses

training loop for task t(After learning task t-1):
  

*   save a copy of the models backbone and prediction/projection head (ft-1 ,gt-1)
*   alongside the cassle predictor ht-1 from previous task


*   your main backbone is the same from previous task

*  the cassle predictor ht is reinitialized

*     Apply two different augmentations(e.g.,v1,v2) to each image, just like standard SSL.

*     Take the representation from one of the augmented views of the current imagePass it through the current CaSSLe predictor network ht (p1)


*     Pass the same representation through the frozen previous task's network and its predictor (q1)


*     compute CaSSLe Distillation Loss typically a cross-entropy or MSE loss
      between q1 adn p1


*     compute the ssl loss for current task using z1 and z2
      combined loss Ltotal

*    The entire current mode (ft,ht,gt) is updated using backpropagation on
     Ltotal


*The main representation learning components (f and g) are carried over.
The CaSSLe predictor from the previous task is saved and frozen as a teacher.
A new CaSSLe predictor is re-initialized for the current task and trained.*

  
  
  











#**MAE**

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from transformers import ViTMAEForPreTraining
from transformers import ViTMAEConfig
from transformers import AutoImageProcessor
from typing import List, Dict, Any
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.optim as optim
from torch.utils.data import ConcatDataset
from PIL import Image
import random
import os
import gc
from torch.cuda.amp import GradScaler, autocast
from transformers import ViTConfig
from transformers import ViTMAEModel


2025-07-26 13:47:58.685072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753537678.948623      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753537679.037238      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


mae finetune on class 1
freeze mae1 and train mae1 on class 2 using ssl loss from mae1 with two augmentations from class 2 and ssl loss from frozen mae1 and g(z) of one aumentatio
then save the trained mae1 as mae2
etc..
so we use the output of the decoder for the ssl loss and use the output of the encoder for the distillation loss

In [3]:

# Custom Dataset for Two Augmentations and Task Filtering
class CustomImageNet100TaskDataset(Dataset):
    def __init__(self, imagenet_dataset, class_list: List[int], transform_v1, transform_v2, processor):
        self.imagenet_dataset = imagenet_dataset
        self.class_set = set(class_list)
        self.valid_indices = []
        self.processor = processor
        
        for i, (_, label) in enumerate(imagenet_dataset.imgs):
            if label in self.class_set:
                self.valid_indices.append(i)
        
        # Make sure transforms don't include ToTensor() or normalization
        # as the processor will handle those
        self.transform_v1 = transform_v1
        self.transform_v2 = transform_v2
    
    def __len__(self):
        return len(self.valid_indices)
    
    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        img_path, original_label = self.imagenet_dataset.imgs[real_idx]
        img = Image.open(img_path).convert('RGB')
        
        # Apply transforms that keep the image as PIL Image
        
        img_v1 = self.transform_v1(img)
        img_v2 = self.transform_v2(img)
        
        # Then apply processor to convert to tensor and normalize
        inputs_v1 = self.processor(images=img_v1, return_tensors="pt")['pixel_values'].squeeze(0)
        inputs_v2 = self.processor(images=img_v2, return_tensors="pt")['pixel_values'].squeeze(0)
        
        return inputs_v1, inputs_v2, original_label
#CaSSLe predictor g(z)
class CaSSLePredictor(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),  # Add BatchNorm
            nn.ReLU(),
            nn.Dropout(0.3),  # Add Dropout for regularization
            nn.Linear(hidden_dim, hidden_dim // 2),  # Additional layer
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim // 2, output_dim),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

class MAECaSSLeModel(nn.Module):
    def __init__(self, mae_backbone: ViTMAEForPreTraining):
        super().__init__()
        self.mae_backbone = mae_backbone
        # The output dimension of the MAE encoder (hidden_size in config)
        self.features_for_h_dim = mae_backbone.config.hidden_size
        #Freezing the decoder
        for param in self.mae_backbone.decoder.parameters():
            param.requires_grad = False


    def forward(self, img_v1: torch.Tensor, img_v2: torch.Tensor) -> Dict[str, Any]:

        # Ensure images have a batch dimension for the MAE model
        # Using output_hidden_states=True to get encoder outputs
        outputs_v1 = self.mae_backbone(pixel_values=img_v1, return_dict=True, output_hidden_states=True)
        outputs_v2 = self.mae_backbone(pixel_values=img_v2, return_dict=True, output_hidden_states=True)

        # For MAE, the last_hidden_state is (batch_size, sequence_length, hidden_size).
        # We average over the sequence length (patches + CLS token) for a single feature vector per image.
        features_for_h_v1 = outputs_v1.hidden_states[-1].mean(dim=1)
        features_for_h_v2 = outputs_v2.hidden_states[-1].mean(dim=1)

        # MAE models often compute the reconstruction loss internally.
        ssl_loss_components_v1 = {'internal_mae_loss': outputs_v1.loss}
        ssl_loss_components_v2 = {'internal_mae_loss': outputs_v2.loss}

        return {
            'features_for_h_v1': features_for_h_v1,
            'features_for_h_v2': features_for_h_v2,
            'ssl_loss_components_v1': ssl_loss_components_v1,
            'ssl_loss_components_v2': ssl_loss_components_v2
        }

    def calculate_ssl_loss(self, ssl_loss_components: Dict[str, Any]) -> torch.Tensor:

        #Extracts and returns the MAE's internal reconstruction loss.
        return ssl_loss_components['internal_mae_loss']

    def get_learnable_params(self) -> List[dict]:
        #Returns the parameters of the entire MAE backbone
        return [{"params": self.mae_backbone.parameters()}]


#training process for each task, handling the current MAE model,
#the frozen previous MAE model, the g predictor, and the two loss terms (L_SSL and L_D).
class CaSSLeTrainer:
    def __init__(self, base_ssl_model: MAECaSSLeModel,
                 ca_predictor_hidden_dim: int,
                 learning_rate: float = 1e-4, lambda_cassle: float =1.0, device: str = 'cuda'):

        self.base_ssl_model = base_ssl_model.to(device) # This is f_t
        self.lambda_cassle = lambda_cassle
        self.device = device

        # Input and output dimensions for CaSSLe Predictor (g) are the MAE encoder's feature dimension
        predictor_g_input_output_dim = self.base_ssl_model.features_for_h_dim

        
        # Initialize the current CaSSLe predictor (g from the paper)
        # It's newly initialized for each task
        self.g_current = CaSSLePredictor(
            predictor_g_input_output_dim,
            ca_predictor_hidden_dim,
            predictor_g_input_output_dim
        ).to(device)

        # Optimizer for ALL trainable parameters: current MAE model (f_t) AND predictor g
        self.optimizer = torch.optim.AdamW(
          self.base_ssl_model.get_learnable_params() + [{"params": self.g_current.parameters()}],
          lr=learning_rate
        )
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=50, eta_min=1e-6
        )

        # This will hold the frozen previous encoder (f_{t-1}^{frozen})
        self.f_frozen_teacher = None

    def set_previous_frozen_encoder(self, encoder_state_dict: Dict[str, Any]):

        # Re-instantiate the MAE model to load the full state dictionary
        self.f_frozen_teacher = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-base').to(self.device)
        self.f_frozen_teacher.load_state_dict(encoder_state_dict)

        # set requires_grad to False for all parameters of the teacher model
        for param in self.f_frozen_teacher.parameters():
            param.requires_grad = False

        print(f"Frozen encoder (f_t-1) loaded and parameters frozen: {all(not p.requires_grad for p in self.f_frozen_teacher.parameters())}")


    def train_task(self, data_loader: torch.utils.data.DataLoader, epochs: int):

        self.base_ssl_model.train()
        self.g_current.train()
        scaler = torch.amp.GradScaler('cuda')  # Initialize GradScaler for AMP
        # Set f_frozen_teacher to eval mode to disable updates for teacher
        if self.f_frozen_teacher:
            self.f_frozen_teacher.eval()

        print(f"Distilling from frozen teacher encoder (f_t-1): {self.f_frozen_teacher is not None}")

        best_loss = float('inf')
        patience = 4
        patience_counter = 0
        min_delta = 0.001

        for epoch in range(epochs):
            total_ssl_loss = 0
            total_cassle_loss = 0
            total_loss = 0

            for batch_idx, (img_v1, img_v2, _) in enumerate(data_loader): # _ for labels
                self.optimizer.zero_grad()
                img_v1 = img_v1.to(self.device)
                img_v2 = img_v2.to(self.device)

                
                self.scaler = GradScaler()
                with autocast():
                  # Forward Pass through the current trainable MAE model (f_t)
                  # This returns features (z_A, z_B) and components for L_SSL
                  ssl_output = self.base_ssl_model(img_v1, img_v2)

                  # Extract features for CaSSLe predictor and APPLY L2 NORMALIZATION HERE
                  features_for_h_v1_norm = F.normalize(ssl_output['features_for_h_v1'], dim=-1)
                  features_for_h_v2_norm = F.normalize(ssl_output['features_for_h_v2'], dim=-1)

                  # Calculate Base Self-Supervised Loss (L_SSL - MAE Reconstruction Loss)
                  # Sum MAE reconstruction loss for both views
                  loss_ssl = self.base_ssl_model.calculate_ssl_loss(ssl_output['ssl_loss_components_v1']) + \
                            self.base_ssl_model.calculate_ssl_loss(ssl_output['ssl_loss_components_v2'])

                  # Calculate CaSSLe Distillation Loss (L_D)
                  loss_cassle = torch.tensor(0.0).to(self.device) # Initialize to 0 for the first task

                  if self.f_frozen_teacher:
                      # Get features from the frozen previous encoder

                      with torch.no_grad():
                        # Get encoder outputs only (bypass the decoder)
                        outputs_v1 = self.f_frozen_teacher.vit(img_v1, output_hidden_states=True)
                        outputs_v1_features= outputs_v1.hidden_states[-1].mean(dim=1)
                        features_from_frozen_v1 = F.normalize(outputs_v1_features, dim=-1)

                        outputs_v2 = self.f_frozen_teacher.vit(img_v2, output_hidden_states=True)
                        outputs_v2_features= outputs_v2.hidden_states[-1].mean(dim=1)
                        features_from_frozen_v2 = F.normalize(outputs_v2_features, dim=-1)

                      # Student predictions from current trainable g
                      # g takes features from current f_t
                      student_pred_v1 = self.g_current(features_for_h_v1_norm)
                      student_pred_v2 = self.g_current(features_for_h_v2_norm)

                      # Teacher targets (from frozen f_t-1)
                      teacher_target_v1 = features_from_frozen_v1
                      teacher_target_v2 = features_from_frozen_v2

                      # Compute distillation loss using MSE
                      loss_cassle_v1 = 1 - F.cosine_similarity(student_pred_v1, teacher_target_v1, dim=-1).mean()
                      loss_cassle_v2 = 1 - F.cosine_similarity(student_pred_v2, teacher_target_v2 ,dim=-1).mean()
                      loss_cassle = (loss_cassle_v1 + loss_cassle_v2 )* self.lambda_cassle

                  # Total Loss and backpropagation
                  loss = loss_ssl +  loss_cassle

                self.scaler.scale(loss).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()

                total_ssl_loss += loss_ssl.item()
                total_cassle_loss += loss_cassle.item()
                total_loss += loss.item()

            #Early stopping
            avg_loss = total_loss / len(data_loader)
            self.scheduler.step()
            if avg_loss < best_loss - min_delta:
                best_loss = avg_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping triggered.")
                    break

            print(f"Epoch {epoch+1}/{epochs} - SSL Loss: {total_ssl_loss / len(data_loader):.4f}, "
                  f"CaSSLe Loss: {total_cassle_loss / len(data_loader):.4f}, "
                  f"Total Loss: {total_loss / len(data_loader):.4f}")

            torch.cuda.empty_cache()


        # After training the task, return the state_dict of the current MAE model (f_t)
        return self.base_ssl_model.mae_backbone.state_dict()


In [4]:

from datasets import load_dataset
class LinearEvalDataset(Dataset):
    """Pre-filter indices for better performance"""
    def __init__(self, original_dataset, class_list, transform):
        if hasattr(original_dataset, 'targets'):
            targets = original_dataset.targets
        else:
            targets = [label for _, label in original_dataset]
        
        # Pre-filter all indices at once
        self.indices = [i for i, label in enumerate(targets) if label in class_list]
        self.original_dataset = original_dataset
        self.transform = transform
        print(f"Filtered dataset: {len(self.indices)} samples from {len(class_list)} classes")
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        img, label = self.original_dataset[self.indices[idx]]
        if self.transform:
            img = self.transform(img)
        return img, label

def extract_features_once(feature_extractor, dataset, batch_size, device):
    """
    Fixed version that handles BatchFeature objects properly.
    """
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4,drop_last=True)
    feature_extractor.eval()
    
    all_features = []
    all_labels = []
        
    with torch.no_grad():
        for batch_data in dataloader:
            if len(batch_data) == 3:
                img_batch, _, label_batch = batch_data
            elif len(batch_data) == 2:
                img_batch, label_batch = batch_data
            else:
                raise ValueError(f"Unexpected batch format: {len(batch_data)} elements")
    
            # Handle HuggingFace BatchFeature or regular tensors
            if hasattr(img_batch, 'pixel_values'):
                pixel_values = img_batch.pixel_values
                if isinstance(pixel_values, list):
                    if isinstance(pixel_values[0], torch.Tensor):
                        img_tensor = torch.stack(pixel_values).to(device)  # ✅ stack list of tensors
                    else:
                        raise TypeError("pixel_values list items are not tensors.")
                else:
                    img_tensor = pixel_values.to(device)
            elif isinstance(img_batch, torch.Tensor):
                img_tensor = img_batch.to(device)
            elif isinstance(img_batch, dict) and 'pixel_values' in img_batch:
                pixel_values = img_batch['pixel_values']
                if isinstance(pixel_values, list):
                    img_tensor = torch.stack(pixel_values).to(device)
                else:
                    img_tensor = pixel_values.to(device)
            else:
                raise TypeError(f"Cannot handle batch type: {type(img_batch)}")
    
            # Ensure tensor has shape [batch_size, 3, H, W]
            if img_tensor.dim() == 5:
                img_tensor = img_tensor.squeeze(0)  # remove extra batch dimension
    
            # Extract features
            output = feature_extractor(img_tensor, output_hidden_states=True)
            features = output.hidden_states[-1][:, 0]  # CLS token
            
            all_features.append(features.cpu())  # Move to CPU to save GPU memory
            all_labels.append(label_batch)
    
    # Concatenate all features and labels
    all_features = torch.cat(all_features, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    
    return all_features, all_labels


# Evaluation Function
def evaluate_model(feature_extractor: torch.nn.Module,
                   all_seen_classes: List[int],
                   train_full, # Full CIFAR100 train dataset
                   val_full,  # Full CIFAR100 test dataset
                   processor, # HuggingFace processor
                   batch_size: int = 128,
                   linear_eval_epochs: int = 15,
                   device: torch.device = torch.device("cuda")):

    # Put feature extractor is in eval mode and frozen
    feature_extractor.eval()
    for param in feature_extractor.parameters():
        param.requires_grad = False

    # Prepare Data for Linear Classifier Training
    # Filter training data to include only classes seen so far
    train_linear_dataset = LinearEvalDataset(train_full, all_seen_classes, processor)
    val_linear_dataset = LinearEvalDataset(val_full, all_seen_classes, processor)

    print("Extracting training features...")
    train_features, train_labels = extract_features_once(feature_extractor, train_linear_dataset, batch_size, device)
    print("Extracting validation features...")
    val_features, val_labels = extract_features_once(feature_extractor, val_linear_dataset, batch_size, device)


    # Map labels to contiguous range
    label_to_contiguous_map = {label: i for i, label in enumerate(sorted(all_seen_classes))}
    train_labels_mapped = torch.tensor([label_to_contiguous_map[l.item()] for l in train_labels])
    val_labels_mapped = torch.tensor([label_to_contiguous_map[l.item()] for l in val_labels])
    
    # Create feature datasets
    train_feature_dataset = TensorDataset(train_features, train_labels_mapped)
    val_feature_dataset = TensorDataset(val_features, val_labels_mapped)
    
    train_loader = DataLoader(train_feature_dataset, batch_size=batch_size, shuffle=True,drop_last=True)
    val_loader = DataLoader(val_feature_dataset, batch_size=batch_size, shuffle=False,drop_last=True)
    
    # Initialize classifier
    features_dim = train_features.shape[1]
    num_output_classes = len(all_seen_classes)
    
    linear_classifier = nn.Sequential(
        nn.Linear(features_dim, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, num_output_classes)
    ).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(linear_classifier.parameters(), lr=1e-3, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=linear_eval_epochs)
    
    # Train classifier (much faster now - no feature extraction)
    linear_classifier.train()
    for epoch in range(linear_eval_epochs):
        for features_batch, labels_batch in train_loader:
            features_batch = features_batch.to(device)
            labels_batch = labels_batch.to(device)
            
            optimizer.zero_grad()
            outputs = linear_classifier(features_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()
        scheduler.step()
    
    # Evaluate
    linear_classifier.eval()
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for features_batch, labels_batch in val_loader:
            features_batch = features_batch.to(device)
            labels_batch = labels_batch.to(device)
            
            outputs = linear_classifier(features_batch)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels_batch.size(0)
            total_correct += (predicted == labels_batch).sum().item()
    
    accuracy = 100 * total_correct / total_samples
    print(f"Linear evaluation accuracy on {len(all_seen_classes)} classes: {accuracy:.2f}%")
    
    # Restore feature extractor
    for param in feature_extractor.parameters():
        param.requires_grad = True
    feature_extractor.train()

    return accuracy
    
# Calculate all random baselines once at the beginning
def calculate_all_random_baselines(task_class_splits, train_full, val_full, processor, device):
    """Calculate random baselines using a randomly initialized ViT-MAE"""
    random_accuracies = {}

    config = ViTMAEConfig()
    random_model = ViTMAEModel(config)
    random_model.to(device)
    random_model.eval()

    try:
        for task_idx, class_list in enumerate(task_class_splits):
            print(f"Calculating random baseline for Task {task_idx+1}")
            random_acc = evaluate_model(
                random_model, class_list, train_full, val_full, processor,
                batch_size=128, linear_eval_epochs=5, device=device
            )
            random_accuracies[task_idx] = random_acc
    finally:
        del random_model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    return random_accuracies

In [ ]:
# --- Configuration ---
# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_SIZE = 224  #  image size
BATCH_SIZE = 16  # 4 rotations × 16 = 66 processed images
NUM_EPOCHS_PER_TASK = 10
LEARNING_RATE = 1e-4
LAMBDA_CASSLE = 0 # Weight for CaSSle loss
NUM_CLASSES_PER_TASK = 10
NUM_TOTAL_CLASSES = 100  # Total  classes
LINEAR_EVAL_EPOCHS = 5
LINEAR_EVAL_BATCH_SIZE = 128
from torchvision import models
torch.cuda.empty_cache()


def set_seed(seed: int = 42):
    import random, os
    import numpy as np
    import torch
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"Random seed set to {seed}")

set_seed(42)

# Load pre-trained MAE-base model and processor
mae_backbone = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-base')
mae_backbone.to(DEVICE)
processor = AutoImageProcessor.from_pretrained('facebook/vit-mae-base',use_fast=True)

# Define the two augmentation pipelines
transform_v1 = transforms.Compose([
    transforms.RandomResizedCrop(INPUT_SIZE, scale=(0.2, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
])

transform_v2 = transforms.Compose([
    transforms.RandomResizedCrop(INPUT_SIZE, scale=(0.2, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.)), # Kernel size must be odd
])

train_full = datasets.ImageFolder(root='/kaggle/input/imagenet100/ImageNet100/train')
val_full = datasets.ImageFolder(root='/kaggle/input/imagenet100/ImageNet100/val')
# Prepare Task Data
# Divide Imagenet-100 into 10 tasks of 10 classes each
task_class_splits = [
    [42, 41, 91, 9, 65, 50, 1, 70, 15, 78],
    [73, 10, 55, 56, 72, 45, 48, 92, 76, 37],
    [30, 21, 32, 96, 80, 49, 83, 26, 87, 33],
    [8, 47, 59, 63, 74, 44, 98, 52, 85, 12],
    [36, 23, 39, 40, 18, 66, 61, 60, 7, 34],
    [99, 46, 2, 51, 16, 38, 58, 68, 22, 62],
    [24, 5, 6, 67, 82, 19, 79, 43, 90, 20],
    [0, 95, 57, 93, 53, 89, 25, 71, 84, 77],
    [64, 29, 27, 88, 97, 4, 54, 75, 11, 69],
    [86, 13, 17, 28, 31, 35, 94, 3, 14, 81]
]
task_datasets = []
for i, class_list in enumerate(task_class_splits):
    print(f"Task {i+1} includes classes: {class_list}")
    task_dataset = CustomImageNet100TaskDataset(train_full, class_list,transform_v1 ,transform_v2,processor)
    task_datasets.append(task_dataset)

# Initialize CaSSLe Components
# base_ssl_model_instance is the current model (f_t) that gets updated across tasks
# It starts with the loaded pretrained MAE.
base_ssl_model_instance = MAECaSSLeModel(mae_backbone=mae_backbone)

# This will store the state_dict of the f_t-1 (previous encoder)
prev_encoder_state_dict = None

print("\n--- Starting CaSSLe Continual Training ---")
print(f"Running on device: {DEVICE}")

# Metrics storage A_j_k will be a list of lists (j tasks observed, k task evaluated on)
# A_j_k[j_idx][k_idx]
all_task_accuracies = [] # Stores accuracies A_{j,k}

print("Calculating random baselines for all tasks...")
random_accuracies_Ri = calculate_all_random_baselines(
    task_class_splits, train_full, val_full, processor, DEVICE
)
print("Random baselines calculated!")


# Continual Training Loop
for task_id, current_task_dataset in enumerate(task_datasets):
    print(f"\n===== Training Task {task_id + 1}/{len(task_datasets)} =====")

    current_task_loader = DataLoader(current_task_dataset, batch_size=BATCH_SIZE, shuffle=True,
                                      num_workers=os.cpu_count() // 2 if os.cpu_count() else 0, pin_memory=True,drop_last=True)


    # Pass the SAME base_ssl_model_instance, as this is the model we continue updating.
    trainer = CaSSLeTrainer(
        base_ssl_model=base_ssl_model_instance,
        ca_predictor_hidden_dim=1024, # Example hidden dim for 'g'
        learning_rate=LEARNING_RATE,
        lambda_cassle=LAMBDA_CASSLE,
        device=DEVICE
    )

    # If it's not the first task, set up the frozen teacher (f_t-1)
    if prev_encoder_state_dict:
        trainer.set_previous_frozen_encoder(prev_encoder_state_dict)

    # Train the current task and get the state of the trained f_t.
    # This will be used as the f_t-1 for the next task.
    prev_encoder_state_dict = trainer.train_task(current_task_loader, NUM_EPOCHS_PER_TASK)

    print(f"\n--- Evaluating after Task {task_id + 1} ---")

    # Determine all classes seen up to this task
    current_seen_classes = sorted(set().union(*task_class_splits[:task_id + 1]))
    accuracies_after_this_task = []

   

    for eval_task_idx in range(task_id + 1):
        # Classes for the specific evaluation task (k)
        eval_task_classes = task_class_splits[eval_task_idx]
        print(f"  Evaluating on classes from Task {eval_task_idx+1}: {eval_task_classes}")

        # Perform linear evaluation on the classes specific to eval_task_idx
        acc_jk = evaluate_model(
            base_ssl_model_instance.mae_backbone, # The current encoder (f_t)
            eval_task_classes, # Evaluate on only classes of task k
            train_full,
            val_full,
            processor,
            LINEAR_EVAL_BATCH_SIZE,
            LINEAR_EVAL_EPOCHS,
            DEVICE
        )
        accuracies_after_this_task.append(acc_jk)

        # Calculate R_i for Forward Transfer only once per task
        random_acc = random_accuracies_Ri[eval_task_idx]
        print(f"    Random baseline for Task {eval_task_idx+1}: {random_acc:.2f}%")
        print(f"    Current accuracy for Task {eval_task_idx+1}: {acc_jk:.2f}%")
        print(f"    Improvement over random: {acc_jk - random_acc:.2f}%")

    all_task_accuracies.append(accuracies_after_this_task)

    # Print current state (optional)
    print(f"\nCurrent A_j,k matrix after Task {task_id + 1}:")
    for j_idx, row in enumerate(all_task_accuracies):
        print(f"After Task {j_idx+1}: {row}")


print("\n CaSSLe Continual Training Process Completed")

# Final Metric Calculation
T = len(task_datasets)

# Average Accuracy
final_accuracies_row = all_task_accuracies[T-1]
avg_accuracy = sum(final_accuracies_row) / T
print(f"\nFinal Average Accuracy (A): {avg_accuracy:.2f}%")


# Forgetting calculation (unchanged)
forgetting = 0
if T > 1:
    for i in range(T - 1):
        max_acc = max(all_task_accuracies[t][i] for t in range(T) if i < len(all_task_accuracies[t]))
        final_acc = all_task_accuracies[T-1][i]
        forgetting += (max_acc - final_acc)
    forgetting /= (T - 1)
print(f"Final Forgetting (F): {forgetting:.2f}%")

# Backward Transfer calculation (unchanged)
backward_transfer = 0
count = 0

if T > 1:
    for new_task in range(1, T):
        for old_task in range(new_task):
            if old_task < len(all_task_accuracies[new_task - 1]) and old_task < len(all_task_accuracies[new_task]):
                acc_before = all_task_accuracies[new_task - 1][old_task]
                acc_after = all_task_accuracies[new_task][old_task]
                backward_transfer += (acc_after - acc_before)
                count += 1
            else:
                print(f"Skipping BT for old_task {old_task+1}, new_task {new_task+1}: missing data")

    backward_transfer /= count if count > 0 else 1
else:
    backward_transfer = 0

print(f"Final Backward Transfer (BT): {backward_transfer:.2f}%")


Random seed set to 42


config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/448M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/217 [00:00<?, ?B/s]

Task 1 includes classes: [42, 41, 91, 9, 65, 50, 1, 70, 15, 78]
Task 2 includes classes: [73, 10, 55, 56, 72, 45, 48, 92, 76, 37]
Task 3 includes classes: [30, 21, 32, 96, 80, 49, 83, 26, 87, 33]
Task 4 includes classes: [8, 47, 59, 63, 74, 44, 98, 52, 85, 12]
Task 5 includes classes: [36, 23, 39, 40, 18, 66, 61, 60, 7, 34]
Task 6 includes classes: [99, 46, 2, 51, 16, 38, 58, 68, 22, 62]
Task 7 includes classes: [24, 5, 6, 67, 82, 19, 79, 43, 90, 20]
Task 8 includes classes: [0, 95, 57, 93, 53, 89, 25, 71, 84, 77]
Task 9 includes classes: [64, 29, 27, 88, 97, 4, 54, 75, 11, 69]
Task 10 includes classes: [86, 13, 17, 28, 31, 35, 94, 3, 14, 81]

--- Starting CaSSLe Continual Training ---
Running on device: cuda
Calculating random baselines for all tasks...
Calculating random baseline for Task 1
Filtered dataset: 12797 samples from 10 classes
Filtered dataset: 500 samples from 10 classes
Extracting training features...
Extracting validation features...
Linear evaluation accuracy on 10 cla

/tmp/ipykernel_36/108681772.py:174: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler()
/tmp/ipykernel_36/108681772.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/10 - SSL Loss: 0.4027, CaSSLe Loss: 0.0000, Total Loss: 0.4027
Epoch 2/10 - SSL Loss: 0.4032, CaSSLe Loss: 0.0000, Total Loss: 0.4032
Epoch 3/10 - SSL Loss: 0.4008, CaSSLe Loss: 0.0000, Total Loss: 0.4008
Epoch 4/10 - SSL Loss: 0.4062, CaSSLe Loss: 0.0000, Total Loss: 0.4062
Epoch 5/10 - SSL Loss: 0.4053, CaSSLe Loss: 0.0000, Total Loss: 0.4053
Epoch 6/10 - SSL Loss: 0.4022, CaSSLe Loss: 0.0000, Total Loss: 0.4022
Early stopping triggered.

--- Evaluating after Task 1 ---
  Evaluating on classes from Task 1: [42, 41, 91, 9, 65, 50, 1, 70, 15, 78]
Filtered dataset: 12797 samples from 10 classes
Filtered dataset: 500 samples from 10 classes
Extracting training features...
Extracting validation features...
Linear evaluation accuracy on 10 classes: 88.54%
    Random baseline for Task 1: 30.47%
    Current accuracy for Task 1: 88.54%
    Improvement over random: 58.07%

Current A_j,k matrix after Task 1:
After Task 1: [88.54166666666667]

===== Training Task 2/10 =====
Frozen encode

In [8]:
torch.save(base_ssl_model_instance.mae_backbone.state_dict(), "ft_task6_encoder.pth")

In [9]:
from copy import deepcopy
# Load previous encoder (f_t from Task 6)
base_ssl_model_instance.mae_backbone.load_state_dict(torch.load("ft_task6_encoder.pth"))
prev_encoder_state_dict = torch.load("ft_task6_encoder.pth")

# Set start task ID
start_task_id = 6  # Task 7 (0-indexed)

# Resume training loop
for task_id, current_task_dataset in enumerate(task_datasets[start_task_id:], start=start_task_id):
    print(f"\n===== Resuming Training: Task {task_id + 1}/{len(task_datasets)} =====")

    current_task_loader = DataLoader(
        current_task_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
        pin_memory=True,
        drop_last=True
    )

    trainer = CaSSLeTrainer(
        base_ssl_model=base_ssl_model_instance,
        ca_predictor_hidden_dim=1024,
        learning_rate=LEARNING_RATE,
        lambda_cassle=LAMBDA_CASSLE,
        device=DEVICE
    )

    trainer.set_previous_frozen_encoder(prev_encoder_state_dict)

    prev_encoder_state_dict = trainer.train_task(current_task_loader, NUM_EPOCHS_PER_TASK)

    # Save model for next resume
    torch.save(base_ssl_model_instance.state_dict(), f"ft_task{task_id + 1}.pth")

    print(f"\n--- Evaluating after Task {task_id + 1} ---")
    current_seen_classes = sorted(set().union(*task_class_splits[:task_id + 1]))
    accuracies_after_this_task = []

    for eval_task_idx in range(task_id + 1):
        eval_task_classes = task_class_splits[eval_task_idx]
        print(f"  Evaluating on classes from Task {eval_task_idx+1}: {eval_task_classes}")

        acc_jk = evaluate_model(
            base_ssl_model_instance.mae_backbone,
            eval_task_classes,
            train_full,
            val_full,
            processor,
            LINEAR_EVAL_BATCH_SIZE,
            LINEAR_EVAL_EPOCHS,
            DEVICE
        )
        accuracies_after_this_task.append(acc_jk)

        random_acc = random_accuracies_Ri[eval_task_idx]
        print(f"    Random baseline for Task {eval_task_idx+1}: {random_acc:.2f}%")
        print(f"    Current accuracy for Task {eval_task_idx+1}: {acc_jk:.2f}%")
        print(f"    Improvement over random: {acc_jk - random_acc:.2f}%")

    all_task_accuracies.append(accuracies_after_this_task)

    # Optional: print current matrix
    print(f"\nCurrent A_j,k matrix after Task {task_id + 1}:")
    for j_idx, row in enumerate(all_task_accuracies):
        print(f"After Task {j_idx+1}: {row}")

# --- Final Metric Calculation ---
T = len(task_datasets)
final_accuracies_row = all_task_accuracies[T-1]
avg_accuracy = sum(final_accuracies_row) / T
print(f"\nFinal Average Accuracy (A): {avg_accuracy:.2f}%")

# Forgetting
forgetting = 0
if T > 1:
    for i in range(T - 1):
        max_acc = max(all_task_accuracies[t][i] for t in range(T) if i < len(all_task_accuracies[t]))
        final_acc = all_task_accuracies[T-1][i]
        forgetting += (max_acc - final_acc)
    forgetting /= (T - 1)
print(f"Final Forgetting (F): {forgetting:.2f}%")

# Backward Transfer
backward_transfer = 0
count = 0
if T > 1:
    for new_task in range(1, T):
        for old_task in range(new_task):
            if old_task < len(all_task_accuracies[new_task - 1]) and old_task < len(all_task_accuracies[new_task]):
                acc_before = all_task_accuracies[new_task - 1][old_task]
                acc_after = all_task_accuracies[new_task][old_task]
                backward_transfer += (acc_after - acc_before)
                count += 1
backward_transfer /= count if count > 0 else 1
print(f"Final Backward Transfer (BT): {backward_transfer:.2f}%")


===== Resuming Training: Task 7/10 =====
Frozen encoder (f_t-1) loaded and parameters frozen: True
Distilling from frozen teacher encoder (f_t-1): True


/tmp/ipykernel_36/821681212.py:174: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler()
/tmp/ipykernel_36/821681212.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50 - SSL Loss: 0.4079, CaSSLe Loss: 0.1562, Total Loss: 0.5640
Epoch 2/50 - SSL Loss: 0.4074, CaSSLe Loss: 0.0764, Total Loss: 0.4838
Epoch 3/50 - SSL Loss: 0.4042, CaSSLe Loss: 0.0613, Total Loss: 0.4655
Epoch 4/50 - SSL Loss: 0.4034, CaSSLe Loss: 0.0530, Total Loss: 0.4564
Epoch 5/50 - SSL Loss: 0.4036, CaSSLe Loss: 0.0474, Total Loss: 0.4510
Epoch 6/50 - SSL Loss: 0.4032, CaSSLe Loss: 0.0445, Total Loss: 0.4477
Epoch 7/50 - SSL Loss: 0.4033, CaSSLe Loss: 0.0418, Total Loss: 0.4451
Epoch 8/50 - SSL Loss: 0.4004, CaSSLe Loss: 0.0400, Total Loss: 0.4403
Epoch 9/50 - SSL Loss: 0.3974, CaSSLe Loss: 0.0387, Total Loss: 0.4361
Early stopping triggered.

--- Evaluating after Task 7 ---
  Evaluating on classes from Task 1: [42, 41, 91, 9, 65, 50, 1, 70, 15, 78]
Filtered dataset: 12797 samples from 10 classes
Filtered dataset: 500 samples from 10 classes
Extracting training features...
Extracting validation features...
Linear evaluation accuracy on 10 classes: 86.80%
    Random baseli